In [1]:
#--------------#
###生成实体类别###
#--------------#

In [2]:
import pickle
from tqdm import tqdm
from prefixspan import PrefixSpan
#import spacy
#import en_core_web_trf
#import spacy_transformers

#nlp = spacy.load('en_core_web_trf')
root_path = '/home/zhangjs/expriment/TKGist/data/'
dataset = 'GDELT/'

from itertools import combinations, chain

def powerset(iterable, num):
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(1, num)))


# read entity2id file
raw_e2id_dict = {}
raw_id2e_dict = {}
file = open(root_path+dataset+'entity2id.txt', 'r')
for e2id in file.readlines():
    e, id = e2id.strip().split('	')[:2]
    raw_e2id_dict[e] = int(id)
    raw_id2e_dict[int(id)] = e
print(len(raw_e2id_dict))
print(raw_id2e_dict[0])

e_num = len(raw_e2id_dict)

# read relation2id file
raw_r2id_dict = {}
raw_id2r_dict = {}
file = open(root_path+dataset+'relation2id.txt', 'r')
for r2id in file.readlines():
    r, id = r2id.strip().split('	')[:2]
    raw_r2id_dict[r] = int(id)
    raw_id2r_dict[int(id)] = r
print(len(raw_r2id_dict))
print(raw_id2r_dict[0])

r_num = len(raw_r2id_dict)
print(r_num)

7691
EGYPT (EGY@ OPP REF LEG SPY MED COP CRM GOV CVL HRI BUS EDU MIL JUD RAD ELI LAB UAF HLH)
240
020
240


In [3]:
file = open(root_path+dataset+'train.txt', 'r')
triple_2_t = dict()
for fact in file.readlines():
    s, r, o, t = fact.strip().split('	')[:4]
    s = int(s)
    r = int(r)
    o = int(o)
    if (s,r,o) not in triple_2_t.keys():
        triple_2_t[(s,r,o)] = []
    triple_2_t[(s,r,o)].append(t)
    if (o,r,s) not in triple_2_t.keys():
        triple_2_t[(o,r,s)] = []
    triple_2_t[(o,r,s)].append(t)

print(len(triple_2_t))
repeat_num = 0
for key in triple_2_t.keys():
    if len(triple_2_t[key]) > 1:
        repeat_num += 1
print(repeat_num)

1267868
542882


In [4]:
# 获取原始关系序列
import random
# read relation sets (注：存在新实体)
seen_e_set = set()
seen_r_set = set()
raw_e2rset_dict = {}
raw_r_type_set = set()
raw_r_type_list = []
raw_e_2_type_list = []
file = open(root_path+dataset+'train.txt', 'r')
for fact in file.readlines():
    s, r, o, t = fact.strip().split('	')[:4]
    s = int(s)
    r = int(r)
    o = int(o)
    t = int(t)
    
    if s not in raw_e2rset_dict.keys():
        raw_e2rset_dict[s] = set()
    if o not in raw_e2rset_dict.keys():
        raw_e2rset_dict[o] = set()
    raw_e2rset_dict[s].add(r)
    raw_e2rset_dict[o].add(r+r_num)

    seen_e_set.add(s)
    seen_e_set.add(o)
    seen_r_set.add(r)
    seen_r_set.add(r+r_num)

for key in raw_e2rset_dict.keys():
    raw_e2rset_dict[key] = sorted(raw_e2rset_dict[key])
    raw_r_type_set.add(tuple(raw_e2rset_dict[key]))
    raw_r_type_list.append(tuple(raw_e2rset_dict[key]))
    raw_e_2_type_list.append(key)
    
print(len(seen_e_set))
print(len(seen_r_set))
print(len(raw_r_type_set))# 原始数据集中有多少种不同的关系组合
print(raw_e2rset_dict)
print(raw_r_type_set)
print(seen_e_set)



7146
476
5591
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 78, 79, 80, 81, 82, 83, 85, 87, 88, 89, 91, 92, 94, 96, 97, 99, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 115, 116, 117, 119, 121, 122, 123, 125, 126, 128, 129, 138, 139, 140, 141, 142, 146, 148, 150, 154, 155, 156, 157, 159, 163, 164, 165, 173, 180, 187, 195, 201, 204, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 318, 319, 320, 321, 322, 323, 324, 326, 327, 328, 329, 331, 

In [5]:
# 获取频繁关系子序列
import numpy as np

tmp_set = set()
cover_e_set = set()
cover_r_set = set()
e_list = raw_e_2_type_list #list(raw_e2rset_dict.keys())
sequence_list = raw_r_type_list #list(raw_e2rset_dict.values())
match_num = [0]*len(sequence_list)
tmp_freq_rset = []
raw_data = []
freq_rset = set()

fail_num = 0
patt_len = 3 # 3 for all datsets

def cover_e(patt, matches):
    cover_e = set()
    for i in matches:
        
        if match_num[i[0]] < 5:
            cover_e.add(e_list[i[0]])
            match_num[i[0]] += 1
        else:
            cover_e.add(e_list[i[0]])
            sequence_list[i[0]] = tuple([])
            match_num[i[0]] += 1
        
    tmp_freq_rset.append((len(cover_e), tuple(sorted(patt)), tuple(sorted(cover_e))))

while len(cover_e_set & seen_e_set) != len(seen_e_set):
    tmp_freq_rset = []
    pre_num_cover_e, pre_num_cover_r = len(cover_e_set), len(cover_r_set)

    ps = PrefixSpan(sequence_list)
    ps.topk(1000, closed = True, filter=lambda patt, matches: len(patt) > patt_len and tuple(sorted(patt)) not in freq_rset, callback=cover_e)
    
    tmp_num_cover_e = pre_num_cover_e
    for rset in tmp_freq_rset:
        if rset[1] not in freq_rset:
            cover_r_set = cover_r_set | set(rset[1])
            cover_e_set = cover_e_set | set(rset[2])
            if len(cover_e_set) > tmp_num_cover_e:
                print([len(cover_e_set), len(cover_r_set)])
                tmp_num_cover_e = len(cover_e_set)
                raw_data.append(rset)
                freq_rset.add(rset[1])

    
    if pre_num_cover_e == len(cover_e_set):
        fail_num += 1
    else:
        pre_num_cover_e = len(cover_e_set)
    if fail_num == 3:
        fail_num = 0
        patt_len = patt_len - 1

print(len(freq_rset))

[2478, 4]
[2696, 6]
[2705, 7]
[2710, 7]
[2870, 7]
[2871, 7]
[2952, 8]
[2953, 8]
[2955, 8]
[2957, 8]
[3036, 8]
[3037, 9]
[3038, 9]
[3055, 9]
[3056, 10]
[3058, 10]
[3059, 10]
[3072, 11]
[3094, 11]
[3105, 12]
[3106, 12]
[3112, 12]
[3130, 12]
[3139, 12]
[3141, 12]
[3162, 12]
[3165, 12]
[3172, 12]
[3178, 12]
[3180, 12]
[3183, 12]
[3251, 12]
[3253, 12]
[3283, 14]
[3287, 14]
[3291, 14]
[3298, 14]
[3305, 14]
[3332, 16]
[3335, 16]
[3347, 16]
[3348, 16]
[3380, 16]
[3382, 16]
[3402, 16]
[3404, 16]
[3410, 16]
[3411, 16]
[3423, 16]
[3426, 16]
[3427, 16]
[3428, 16]
[3429, 16]
[3494, 16]
[3496, 16]
[3497, 16]
[3500, 16]
[3501, 16]
[3504, 16]
[3511, 16]
[3519, 16]
[3528, 16]
[3531, 16]
[3537, 16]
[3539, 16]
[3540, 16]
[3548, 16]
[3552, 16]
[3553, 16]
[3554, 16]
[3560, 16]
[3561, 16]
[3562, 16]
[3563, 16]
[3568, 16]
[3581, 16]
[3588, 16]
[3596, 16]
[3598, 16]
[3601, 16]
[3602, 16]
[3615, 16]
[3621, 16]
[3623, 16]
[3624, 16]
[3625, 16]
[3629, 16]
[3630, 16]
[3634, 16]
[3635, 16]
[3636, 16]
[3637, 16]
[3

In [6]:
# 基于集合包含关系合并频繁关系子序列

# 基于类别的集合包含关系合并 O(N*2^k), 优化前为 O(N^2), 2^k << N
sorted_freq_r_list = sorted(raw_data, key = lambda x:len(x[1]), reverse = True)

sparse_type_set = set()
childset_2_fatherset = {}
cover_e_2_freq_r_dict = {}
filtered_freq_r_2_cover_e_dict = {}
for sub_r_set in tqdm(sorted_freq_r_list):
    patt = tuple(sorted(sub_r_set[1]))
    cover_e = sub_r_set[2]
    if 'YAGO' in dataset:
        filtered_freq_r_2_cover_e_dict[patt] = tuple(sorted(cover_e))
        continue

    if patt in childset_2_fatherset.keys():
        fathersets = childset_2_fatherset[patt]
        for father_set in fathersets:
            filtered_freq_r_2_cover_e_dict[father_set] = tuple(sorted(set(filtered_freq_r_2_cover_e_dict[father_set]) | set(cover_e)))
    
    else:
        if len(cover_e) >= 5:
            filtered_freq_r_2_cover_e_dict[patt] = tuple(sorted(cover_e))
            childsets = powerset(patt, len(patt)+1)
            for child_set in childsets:
                child_set = tuple(sorted(child_set))
                if child_set not in childset_2_fatherset.keys():
                    childset_2_fatherset[child_set] = []
                childset_2_fatherset[child_set].append(patt)
        else:
            childsets = powerset(patt, len(patt))
            flag = 0
            for child_set in childsets:
                child_set = tuple(sorted(child_set))
                if child_set in childset_2_fatherset.keys():
                    flag = 1
                    fathersets = childset_2_fatherset[child_set]
                    for father_set in fathersets:
                        filtered_freq_r_2_cover_e_dict[father_set] = tuple(sorted(set(filtered_freq_r_2_cover_e_dict[father_set]) | set(cover_e)))
            if flag == 0:
                filtered_freq_r_2_cover_e_dict[patt] = tuple(sorted(cover_e))
                if len(cover_e) == 1:
                    sparse_type_set.add(patt)

print("Number of sparse types: " + str(len(sparse_type_set)))
print("Sparse types: " + str(sparse_type_set))

for key in filtered_freq_r_2_cover_e_dict.keys():
    value = tuple(sorted(filtered_freq_r_2_cover_e_dict[key]))
    if value not in cover_e_2_freq_r_dict.keys():
        cover_e_2_freq_r_dict[value] = tuple(sorted(key))
    else:
        cover_e_2_freq_r_dict[value] = tuple(sorted(cover_e_2_freq_r_dict[value] | set(key)))

print("Number of types: " + str(len(cover_e_2_freq_r_dict.keys())))

if 'YAGO' not in dataset:
    cover_e_set = set()
    for key in cover_e_2_freq_r_dict.keys():
        cover_e_set = cover_e_set | set(key)
    print(len(cover_e_set & seen_e_set))

    # 基于覆盖实体的集合包含关系合并, 优化前为 O(M^2)
    del_num = 1
    times = 0
    while del_num != 0 and times < 5:
        times += 1
        sorted_cover_e_list = sorted(cover_e_2_freq_r_dict.items(), key = lambda x:len(x[0]), reverse = False)

        del_set = set()
        add_dict = {} #father_cover_e 2 (child_cover_e, r_of_child_cover_e)
        for i in tqdm(range(len(sorted_cover_e_list) - 1)):
            cover_e = sorted_cover_e_list[i][0]
            patt = sorted_cover_e_list[i][1]
            in_i = i+1
            while in_i < len(sorted_cover_e_list):
                in_cover_e = sorted_cover_e_list[in_i][0]
                in_patt = sorted_cover_e_list[in_i][1]
                in_i += 1

                cover_ratio = float(len(set(cover_e) & set(in_cover_e))) / float(min(len(cover_e), len(in_cover_e)))
                if cover_ratio >= 0.8: # 0.8
                    if in_cover_e not in add_dict.keys():
                        add_dict[in_cover_e] = []
                    add_dict[in_cover_e].append([set(cover_e), set(patt)])
                    del_set.add(cover_e)
                else:
                    if len(cover_e) < 5 and len(set(cover_e) - set(in_cover_e)) < len(cover_e):
                        if in_cover_e not in add_dict.keys():
                            add_dict[in_cover_e] = []
                        add_dict[in_cover_e].append([set(cover_e), set(patt)])
                        del_set.add(cover_e)

        del_num = 0
        cover_e_2_freq_r_dict = {}
        for item in sorted_cover_e_list:
            cover_e = item[0]
            patt = item[1]

            if cover_e in add_dict.keys():
                new_cover_e = set(cover_e)
                new_patt = set(patt)
                for in_item in add_dict[cover_e]:
                    new_cover_e = new_cover_e | in_item[0]
                    new_patt = new_patt | in_item[1]
                new_cover_e = tuple(sorted(new_cover_e))
                new_patt = tuple(sorted(new_patt))
                cover_e_2_freq_r_dict[new_cover_e] = new_patt
            else:
                if cover_e in del_set:
                    del_num += 1
                    continue
                else:
                    cover_e_2_freq_r_dict[tuple(cover_e)] = tuple(patt)
    
        print("DEL " + str(del_num) + " types")
        cover_e_set = set()
        for key in cover_e_2_freq_r_dict.keys():
            cover_e_set = cover_e_set | set(key)
        print([len(cover_e_set & seen_e_set), len(cover_e_set), len(seen_e_set)])
                

    # 获取类别—>覆盖实体 字典

    filtered_freq_r_2_cover_e_dict = {}
    for cover_e in cover_e_2_freq_r_dict.keys():
        patt = cover_e_2_freq_r_dict[cover_e]
        if patt not in filtered_freq_r_2_cover_e_dict.keys():
            filtered_freq_r_2_cover_e_dict[patt] = cover_e
        else:
            filtered_freq_r_2_cover_e_dict[patt] = tuple(sorted(set(cover_e) | set(filtered_freq_r_2_cover_e_dict[patt])))

print("Number of filtered types: " + str(len(filtered_freq_r_2_cover_e_dict.keys())))
   

100%|██████████| 1866/1866 [00:11<00:00, 161.76it/s]


Number of sparse types: 226
Sparse types: {(16, 22, 28, 37, 55, 67, 264), (0, 13, 30, 260, 266, 274), (7, 23, 129, 269, 271, 274), (7, 15, 18, 24, 44, 45), (7, 11, 114, 240, 245, 256, 264, 299, 316, 323), (3, 15, 113, 115, 242, 353, 402), (6, 53, 110, 256, 258, 260, 262, 277), (23, 69, 309, 314, 329, 337), (166, 342, 350), (6, 22, 246, 252, 268, 342), (2, 32, 240, 243, 278, 292), (9, 37, 247, 256, 267, 299, 369), (25, 253, 265, 267, 299), (15, 65, 257, 263, 277), (16, 37, 80, 244, 276), (0, 1, 59, 112, 299), (2, 57, 240, 243, 297), (6, 245, 246, 252, 260), (23, 28, 60, 80, 115, 136, 263, 278, 281, 300, 307, 329), (7, 19, 36, 244, 259, 277), (11, 24, 89, 127, 164, 251, 258, 274, 276, 277, 313, 387), (7, 24, 36, 240, 256), (0, 13, 16, 27, 65, 268), (6, 16, 41, 252, 256, 257), (56,), (7, 17, 18, 63, 303, 321, 363), (22, 23, 51, 65, 293), (7, 18, 30, 246, 296), (240, 244, 246, 258, 263, 271), (11, 18, 52, 256, 273, 335), (18, 52, 247, 264, 292), (15, 17, 22, 47, 256, 312), (38, 44, 253), (

100%|██████████| 680/680 [00:18<00:00, 37.05it/s]


DEL 189 types
[7146, 7146, 7146]


100%|██████████| 491/491 [00:22<00:00, 21.83it/s]


...

DEL 20 types
[7146, 7146, 7146]


100%|██████████| 381/381 [00:18<00:00, 21.15it/s]


DEL 29 types
[7146, 7146, 7146]


100%|██████████| 326/326 [00:12<00:00, 26.76it/s]


DEL 18 types
[7146, 7146, 7146]
Number of filtered types: 220


In [7]:
print(filtered_freq_r_2_cover_e_dict)

cover_e_set = set()
for value in filtered_freq_r_2_cover_e_dict.values():
    cover_e_set = cover_e_set | set(value)
print(len(cover_e_set & seen_e_set))

{(11, 24, 89, 127, 164, 251, 258, 274, 276, 277, 313, 387): (2248,), (7, 11, 114, 240, 245, 256, 264, 299, 316, 323): (2428,), (7, 18, 22, 37, 46, 71, 252, 263, 284): (3784,), (0, 10, 83, 240, 250, 259, 271, 277): (3023,), (6, 17, 65, 73, 93, 137, 240, 257): (3920,), (7, 24, 36, 44, 241, 267, 279, 284): (5303,), (23, 156, 160, 250, 298, 387, 396, 400): (4855,), (28, 29, 52, 247, 256, 268, 292, 307): (5406,), (3, 31, 242, 244, 255, 286, 364): (6521,), (16, 22, 28, 37, 55, 67, 264): (3791,), (20, 27, 44, 108, 240, 247, 284): (3345,), (20, 106, 240, 246, 264, 274, 292): (3497,), (20, 246, 260, 263, 271, 274, 359): (3639,), (24, 255, 256, 260, 264, 268, 307): (3984,), (0, 9, 27, 268, 276, 353): (5836,), (0, 13, 30, 260, 266, 274): (5145,), (0, 16, 29, 244, 247, 274): (3312,), (0, 244, 252, 257, 269, 274): (5416,), (6, 22, 246, 252, 268, 342): (1745,), (7, 10, 16, 24, 240, 250): (3769,), (7, 12, 16, 28, 30, 37): (2058,), (7, 15, 18, 24, 44, 45): (4160,), (9, 11, 59, 247, 256, 268): (2832,),

In [8]:
# 基于贪心算法求最小集合覆盖
uncover_e_set = seen_e_set
final_r_types = set()
iter_num = 1

while len(uncover_e_set) > 0:
    print("iter" + str(iter_num) + "|" + str(len(final_r_types)) + "|" + str(len(uncover_e_set)))
    iter_num += 1
    max_cover_num = 0
    max_cover_type = None
    max_cover_e = set()
    for r_type in filtered_freq_r_2_cover_e_dict.keys():
        inject_num = len(uncover_e_set & set(filtered_freq_r_2_cover_e_dict[r_type]))
        if inject_num > max_cover_num:
            max_cover_num = inject_num
            max_cover_type = r_type
            max_cover_e = set(filtered_freq_r_2_cover_e_dict[r_type])
    if max_cover_num != 0:
        uncover_e_set = uncover_e_set - max_cover_e
        final_r_types.add(max_cover_type)

print("Final number of types: " + str(len(final_r_types)))

iter1|0|7146
iter2|1|207
iter3|2|197
iter4|3|190
iter5|4|183
iter6|5|176
iter7|6|169
iter8|7|163
iter9|8|157
iter10|9|152
iter11|10|147
iter12|11|142
iter13|12|138
iter14|13|134
iter15|14|130
iter16|15|126
iter17|16|122
iter18|17|118
iter19|18|114
iter20|19|110
iter21|20|107
iter22|21|104
iter23|22|101
iter24|23|98
iter25|24|95
iter26|25|92
iter27|26|89
iter28|27|86
iter29|28|84
iter30|29|82
iter31|30|80
iter32|31|78
iter33|32|76
iter34|33|74
iter35|34|72
iter36|35|70
iter37|36|68
iter38|37|66
iter39|38|65
iter40|39|64
iter41|40|63
iter42|41|62
iter43|42|61
iter44|43|60
iter45|44|59
iter46|45|58
iter47|46|57
iter48|47|56
iter49|48|55
iter50|49|54
iter51|50|53
iter52|51|52
iter53|52|51
iter54|53|50
iter55|54|49
iter56|55|48
iter57|56|47
iter58|57|46
iter59|58|45
iter60|59|44
iter61|60|43
iter62|61|42
iter63|62|41
iter64|63|40
iter65|64|39
iter66|65|38
iter67|66|37
iter68|67|36
iter69|68|35
iter70|69|34
iter71|70|33
iter72|71|32
iter73|72|31
iter74|73|30
iter75|74|29
iter76|75|28
iter77|

In [9]:
# 通过约束每个实体的最大类别数消减类别数量
import numpy as np

k = 5 #10 for ICEWS14 ,ICEWS05, YAGO
e_2_type_dict = {}
type_2_e_dict = {}

for key in filtered_freq_r_2_cover_e_dict.keys():
    for e in filtered_freq_r_2_cover_e_dict[key]:
        if e not in e_2_type_dict.keys():
            e_2_type_dict[e] = set()
        e_2_type_dict[e].add(key)

for e in e_2_type_dict.keys():
    old_types = e_2_type_dict[e]
    sorted_old_types = sorted(old_types, key=lambda type: len(filtered_freq_r_2_cover_e_dict[type]), reverse=True)
    new_types = sorted_old_types[:k]
    e_2_type_dict[e] = new_types
    for type in new_types:
        if type not in type_2_e_dict.keys():
            type_2_e_dict[type] = set()
        type_2_e_dict[type].add(e)

r_2_cover_e_num = []
e_2_type_num = []
for key in type_2_e_dict.keys():
    r_2_cover_e_num.append(len(type_2_e_dict[key]))
for key in e_2_type_dict.keys():
    e_2_type_num.append(len(e_2_type_dict[key]))

print(len(type_2_e_dict.keys()))

# 关系的覆盖实体数量统计
print([min(r_2_cover_e_num), np.mean(r_2_cover_e_num), max(r_2_cover_e_num)])

# 实体的关系数量统计
print([min(e_2_type_num), np.mean(e_2_type_num), max(e_2_type_num)])


127
[1, 217.53543307086613, 6939]
[1, 3.86607892527288, 5]


In [10]:
print(type_2_e_dict)

{(11, 24, 89, 127, 164, 251, 258, 274, 276, 277, 313, 387): {2248}, (7, 11, 114, 240, 245, 256, 264, 299, 316, 323): {2428}, (7, 18, 22, 37, 46, 71, 252, 263, 284): {3784}, (0, 10, 83, 240, 250, 259, 271, 277): {3023}, (6, 17, 65, 73, 93, 137, 240, 257): {3920}, (7, 24, 36, 44, 241, 267, 279, 284): {5303}, (23, 156, 160, 250, 298, 387, 396, 400): {4855}, (28, 29, 52, 247, 256, 268, 292, 307): {5406}, (3, 31, 242, 244, 255, 286, 364): {6521}, (16, 22, 28, 37, 55, 67, 264): {3791}, (20, 27, 44, 108, 240, 247, 284): {3345}, (20, 106, 240, 246, 264, 274, 292): {3497}, (20, 246, 260, 263, 271, 274, 359): {3639}, (24, 255, 256, 260, 264, 268, 307): {3984}, (0, 9, 27, 268, 276, 353): {5836}, (0, 13, 30, 260, 266, 274): {5145}, (0, 16, 29, 244, 247, 274): {3312}, (0, 244, 252, 257, 269, 274): {5416}, (6, 22, 246, 252, 268, 342): {1745}, (7, 10, 16, 24, 240, 250): {3769}, (7, 12, 16, 28, 30, 37): {2058}, (7, 15, 18, 24, 44, 45): {4160}, (9, 11, 59, 247, 256, 268): {2832}, (10, 11, 15, 17, 267, 

In [11]:
# 存储类别文件
import numpy as np

type_2_id = dict()
id = 0

f = open(root_path+dataset+'types.txt','w', encoding='utf-8')
for type in type_2_e_dict.keys():
    type_2_id[type] = id
    f.writelines(str(type) + '	' + str(id) + '\n')
    id += 1
f.close()

e_2_type_id = dict()
f = open(root_path+dataset+'e2types.txt','w', encoding='utf-8')
for e in e_2_type_dict.keys():
    types = e_2_type_dict[e]
    ids = [type_2_id[type] for type in types]
    e_2_type_id[e] = ids
    f.writelines(str(e) + '	' + str(ids) + '\n')
f.close()

print(len(type_2_id.items()))


127
